# The KLT Tracker
*By Erik Gärtner*


### 1.2.2 Inverse-additive scheme

The inverse-additive shceme is the following:
\begin{align}
E(u, v) = \sum_{x,y} [J(x, y) - I(x - u, y -v)] ^2
\end{align}

Which is derived into the form of $Zd = e$:

\begin{align}
\sum_{x,y} 
\begin{bmatrix}
I_x^2 & I_x I_y \\
I_x I_y & I_y^2 \\
\end{bmatrix} 
\begin{bmatrix}
u \\
v
\end{bmatrix} = - \sum_{x,y}
\begin{bmatrix}
I_x D \\
I_y D
\end{bmatrix}
\end{align}

where D is $D(x,y) = J(x,y) - I(x - u,y -v)$

### Helper functions

In [275]:
import numpy as np
from  scipy import ndimage, interpolate

from matplotlib.pyplot import imshow, imread
import matplotlib.pyplot as plt
from PIL import Image

In [276]:
def load_img(path='./view0.png', gray=True):
    rgb = imread(path)
    rgb = np.array(rgb, dtype=np.float32)
    if gray:
        return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])
    else:
        return rgb

I = load_img('./view0.png')
J = load_img('./view1.png')


### 1.3.1 Gradient Function

In [277]:
# Sharr filter
#X_FILTER = np.array([[3.0, 0.0, -3.0], [10.0, 0.0, 10.0], [3.0, 0.0, -3.0]])
# Sobel filter
X_FILTER = np.array([[1.0, 0.0, -1.0], [2., 0., -2.], [1.0, 0.0, -1.0]])
Y_FILTER = X_FILTER.T

def differentiate(img_arr):
    x_diff = ndimage.convolve(img_arr, X_FILTER)#, mode='constant', cval=0.0)
    y_diff = ndimage.convolve(img_arr, Y_FILTER)#, mode='constant', cval=0.0)
    return (x_diff, y_diff)

### 1.3.2 Estimating Z
Z is the Hessian matrix.

In [278]:
def estimate_Z(x_diff, y_diff):
    x_diff = x_diff.flatten()
    y_diff = y_diff.flatten()
    
    grads_T = np.array([x_diff, y_diff])
    Z = np.dot(grads_T, grads_T.T)
    return Z

### 1.3.3 Difference Function

In [279]:
def estimate_e(I, J, Ix, Iy):   
    D = (I - J).flatten()
    Ix = Ix.flatten()
    Iy = Iy.flatten()
    
    grads_T = np.array([Ix, Iy])
    
    return -1 *np.dot(grads_T, D)

### 1.3.4 Interpolation Function

In [300]:
def interpolate_region(img_array):
    """
    Creates an bilinear interpolation of the image area sent in.
    Evaluate using spline(x, y)
    """
    x_arr = np.arange(0, img_array.shape[0])
    y_arr = np.arange(0, img_array.shape[1])
    spline = interpolate.RectBivariateSpline(x_arr, y_arr, img_array, kx=1, ky=1)
    return spline

### 1.3.5 Finalizing the KLT Tracker

Some notes, 
- I had to use the pseudo-inverse because I got a singular matrix Z.
- Should I interpolate the images as I have done, that is first compute the gradients of the entire image, then create interpolation of the entire image and image gradients.

In [301]:
def calc_d(I, J, x, y, win_size, max_iter, min_disp):
    
    it = 0
    d_tot = np.array([0., 0.]).T;
    
    # The window to evaluate
    win_x = np.arange(x, x + win_size[0], dtype=float)
    win_y = np.arange(y, y + win_size[1], dtype=float)
    
    template = I[x:x + win_size[0], y: y + win_size[1]]
    
    # Find image gradient in I
    Ix, Iy = differentiate(template)
    print(Ix)
    
    Z = estimate_Z(Ix, Iy)
    Zinv = np.linalg.inv(Z)
    print(Zinv)
    
    # Create interpolated versions of gradient and images
    I_inter = interpolate_region(I)
    J_inter = interpolate_region(J)  
    
    while it < max_iter:
        it += 1
        
        # Get the current window
        J_win = J_inter(win_x, win_y)

        e = estimate_e(template, J_win, Ix, Iy)       
        d = np.dot(Zinv, e)
        
        print(d_tot, d)
        d_tot = d_tot + d
        
        if np.hypot(d[0], d[1]) <= min_disp:
            # Check if converged
            return d_tot
        
        # Shift I and J by d
        win_x = win_x + d[0]
        win_y = win_y + d[1]
        
    return d_tot   

In [302]:
def calc_klt(old_image, new_image, input_points, win_size=(21, 21), max_iter=30, min_disp=0.01):
    
    output_points = []
    for (x, y) in input_points:
        d = calc_d(old_image, new_image, x, y, win_size, max_iter, min_disp)
        output_points.append((x + d[0], y + d[1]))
    return output_points

### 1.3.6 Test Implementation

In [264]:
import cv2

def opencv_klt(old_image, new_image, input_points, win_size=(21, 21)):
    pts = np.array(input_points ,np.float32)
    I2 = I.astype(np.uint8)
    J2 = J.astype(np.uint8)
    res = cv2.calcOpticalFlowPyrLK(I2, J2, pts, None, winSize=win_size, maxLevel=0) #prevPts[, nextPts[, status[, err[, winSize[, maxLevel[, criteria[, flags[, minEigThreshold]]]]]]]])
    return res[0]

In [265]:
#imshow(I, cmap = plt.get_cmap('gray'))

In [303]:
calc_klt(I, J, [(320, 336)], (100, 100))

[[ 6.66666627e-05  1.40784305e-03  0.00000000e+00 ... -8.18823481e-03
  -1.30039208e-02 -3.92156839e-03]
 [ 3.77647036e-03  4.22352916e-03  0.00000000e+00 ... -1.23921561e-03
  -1.08705876e-02 -7.84313679e-03]
 [ 4.22352916e-03  4.22352916e-03  0.00000000e+00 ... -5.13725460e-04
   1.11022302e-16  1.11022302e-16]
 ...
 [-3.47450960e-03 -3.98823506e-03 -3.92156839e-03 ... -6.35529442e-02
  -9.49254845e-02 -5.49019575e-02]
 [ 0.00000000e+00 -2.36862731e-03  0.00000000e+00 ... -2.77019625e-02
  -3.04941158e-02 -1.45568619e-02]
 [ 0.00000000e+00 -1.40784305e-03  0.00000000e+00 ...  1.10666660e-02
   3.79843115e-02  3.47607822e-02]]
[[0.09624907 0.01526949]
 [0.01526949 0.02270437]]
[0. 0.] [ 0.10838337 -0.05417738]
[ 0.10838337 -0.05417738] [ 0.11128783 -0.05048359]
[ 0.2196712  -0.10466097] [ 0.11412251 -0.04668439]
[ 0.33379371 -0.15134536] [ 0.11688525 -0.04278519]
[ 0.45067896 -0.19413055] [ 0.11957425 -0.03879181]
[ 0.57025321 -0.23292236] [ 0.12218802 -0.03471051]
[ 0.69244123 -0.267

[(323.6259101365383, 336.1253067784293)]

In [304]:
opencv_klt(I, J, [(320, 336)], (21, 21))

array([[320., 336.]], dtype=float32)

##### Compare Gradients

In [299]:
I = load_img('./alt/view0.png')
J = load_img('./alt/view1.png')
I[213:265,328:419].shape

(52, 91)